In [1]:
import underworld as uw
import numpy as np

In [2]:
mesh = uw.mesh.FeMesh_Cartesian()

In [3]:
swarm = uw.swarm.Swarm(mesh)
def addvars(swarm_inst):
    return (
            ( swarm_inst.add_variable('char'  ,1), np.iinfo(np.int8).max    , 'char'   ),
            ( swarm_inst.add_variable('short' ,1), np.iinfo(np.int16).max   , 'short'  ),
            ( swarm_inst.add_variable('int'   ,1), np.iinfo(np.int32).max   , 'int'    ),
            ( swarm_inst.add_variable('long'  ,1), np.iinfo(np.int64).max   , 'long'   ),
            ( swarm_inst.add_variable('float' ,1), np.finfo(np.float32).max , 'float'  ),
            ( swarm_inst.add_variable('double',1), np.finfo(np.float64).max , 'double' ) )
vars = addvars(swarm)

In [4]:
swarm.populate_using_layout(uw.swarm.layouts.PerCellSpaceFillerLayout(swarm,20))

In [5]:
swarm.save("test_swarm.h5")
for var in vars:
    var[0].data[:] = var[1]
    var[0].save("test_swarvar_" + var[2] + ".h5")

In [6]:
swarm2 = uw.swarm.Swarm(mesh)
vars2 = addvars(swarm2)

In [7]:
swarm2.load("test_swarm.h5")
for var in vars2:
    var[0].load("test_swarvar_" + var[2] + ".h5")
    if not np.allclose(var[0].data, var[1] ):
        raise RuntimeError("Something went wrong with swarm variable object.")

In [8]:
if uw.mpi.rank == 0: 
    import os
    os.remove( "test_swarm.h5" )
    for var in vars:
        os.remove( "test_swarvar_" + var[2] + ".h5" ) 